<a href="https://colab.research.google.com/github/buupq/torch/blob/main/tinyVGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# create sources directory
from pathlib import Path

sources = Path("sources")
sources.mkdir(parents=True, exist_ok=True)

In [19]:
%%writefile sources/utils.py
import os
from pathlib import Path
import torch

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def list_file_tree(
    targ_dir=".",
    print_all_files=False,
    num_file_cap=2,
    print_dir_only=True):

    """list directories and files of a target directory
    Args:
        targ_dir: target directory
        print_all_files: print all available directory and files in targ_dir
        print_dir_only: print directory tree only
        num_file_cap: maximum number of files listed
    Examples:
        list_file_tree(targ_dir=".", print_dir_only=True)
        would return only list of directory
    """
    if not Path(targ_dir).is_dir():
        return f"{targ_dir}: does not exist!"

    for root, dirs, files in list(os.walk(targ_dir)):
        # print directories
        level = root.replace(str(targ_dir), "").count(os.sep)
        indent = " " * 3 * level
        print(f"{indent} {bcolors.OKBLUE} {root} {bcolors.ENDC}")

        if not print_dir_only:
        # print files in directory
            subindent = " " * 3 * (level + 1)
            num_files = 0
            if print_all_files:
                num_file_cap = len(files)
            for file in files:
                if num_files < num_file_cap:
                    print(f"{subindent} {bcolors.OKGREEN} {file} {bcolors.ENDC}")
                    num_files += 1
                    if num_files == num_file_cap:
                        print(f"{subindent} and {len(files)} other files...")


# download data
from pathlib import Path
from zipfile import ZipFile
import requests

def download_data(url):
    """Download file from an url to data, extract files to images folder
    Args:
        url: url to the data source
    Returns:
        image_path, train_path, test_path = download_data(url=url)
            image_path: folder to store train and test images
            train_path: folder to store train data
            test_path: folder to store test data
    """

    # create data directory
    data_path = Path("data")
    image_path = data_path / "images"
    image_path.mkdir(parents=True, exist_ok=True)

    # download zip file to data/
    request = requests.get(url)
    with open(data_path / "images.zip", "wb") as f:
        f.write(request.content)

    # extract images to data/images
    with ZipFile(data_path / "images.zip") as zip_ref:
        zip_ref.extractall(image_path)

    # train and test directories
    train_dir = image_path / "train"
    test_dir = image_path / "test"


    return image_path, train_dir, test_dir


def save_model(model:torch.nn.Module,
               model_dir: str):

    model_save_path = Path(model_dir) / (model.name + ".pth")

    torch.save(
        obj = model.state_dict(),
        f = model_save_path
    )

    return model_save_path

def load_saved_model(loaded_model: torch.nn.Module,
                     model_saved_path: str):

    loaded_model.load_state_dict(torch.load(model_saved_path))


Overwriting sources/utils.py


In [3]:
%%writefile sources/datasetup.py
# create train and test dataloader
#     using ImageFolder
#     using DataLoader

import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

def create_compose_transforms():
    train_transform = transforms.Compose([
        transforms.Resize(size = (64, 64)),
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(p=0.5)
    ])

    test_transform = transforms.Compose([
        transforms.Resize(size=(64,64)),
        transforms.ToTensor()
    ])

    return train_transform, test_transform


def create_dataloaders(train_dir: str,
                    test_dir: str,
                    train_transform: torchvision.transforms.Compose,
                    test_transform: torchvision.transforms.Compose,
                    batch_size: int,
                    num_workers: int):

    # create train data from image folder
    train_data = ImageFolder(
        root = train_dir,
        transform=train_transform,
        target_transform=None
    )

    # create test data from image folder
    test_data = ImageFolder(
        root = test_dir,
        transform = test_transform,
        target_transform = None
    )

    # get train data class names
    class_names = train_data.classes

    # create train dataloader
    train_dataloader = DataLoader(
        train_data,
        batch_size = batch_size,
        num_workers = num_workers,
        shuffle = True
    )

    # create test dataloader
    test_dataloader = DataLoader(
        test_data,
        batch_size = batch_size,
        num_workers = num_workers
    )

    return train_dataloader, test_dataloader, class_names


Overwriting sources/datasetup.py


In [4]:
# BATCHSIZE = 32
# NUMWORKERS = 1

# from torchvision import transforms

# train_transform = transforms.Compose([
#     transforms.Resize(size=(64, 64)),
#     transforms.ToTensor()
# ])

# test_transform = transforms.Compose([
#     transforms.Resize(size=(64, 64)),
#     transforms.ToTensor()
# ])

# train_dataloader, test_dataloader, class_names = create_dataloaders(
#     train_dir= train_dir,
#     test_dir = test_dir,
#     train_transform = train_transform,
#     test_transform = test_transform,
#     batch_size = BATCHSIZE,
#     num_workers=NUMWORKERS
# )

# len(train_dataloader), len(test_dataloader)
# img_batch, label_batch = next(iter(train_dataloader))
# img_batch.shape, label_batch.shape


In [5]:
%%writefile sources/engine.py

# main engine
import torch
from torch import nn

class tinyVGG(nn.Module):
    def __init__(self, name: str, inp_shape: int, out_shape: int, hidden_units=10):
        super().__init__()

        self.name = name

        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=inp_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*16*16, out_features=out_shape)
        )

    def forward(self, x: torch.Tensor):
        return self.classifier(self.block_2(self.block_1(x)))

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):

    # switch to train mode
    model.train()

    # initialize train loss and accuracy
    train_loss, train_acc = 0, 0

    # loop over dataloader batch
    for batch, (X, y) in enumerate(dataloader):

        # compute train logit
        train_logit = model(X)

        # compute loss
        loss = loss_fn(train_logit, y)

        # accumulate train loss
        train_loss += loss.item()

        # zero optimizer gradient
        optimizer.zero_grad()

        # back propagation
        loss.backward()

        # optimizer steps
        optimizer.step()

        # predict image labels
        y_pred = torch.argmax(torch.softmax(train_logit, dim=1), dim=1)

        # compute and accumulate accuracy based on predicted labels
        train_acc += (y_pred == y).sum().item() / len(y_pred)

    # adjust train loss and accuracy
    train_loss /= len(dataloader)
    train_acc /= len(dataloader)

    # return train loss and accuracy
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):

    # switch to evaluation mode
    model.eval()

    # initialize test loss and accuracy
    test_loss, test_acc = 0, 0

    # loop over dataloader batch
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):

            # compute test logit
            test_logit = model(X)

            # compute and accumulate test loss
            test_loss += loss_fn(test_logit, y).item()

            # compute test labels
            y_pred = torch.argmax(torch.softmax(test_logit, dim=1), dim=1)

            # compute and accumulate test accuracy
            test_acc += (y_pred == y).sum().item() / len(y_pred)

        # adjust test loss and accuracy
        test_loss /= len(dataloader)
        test_acc /= len(dataloader)

    # return test loss and accuracy
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int=3):


    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []}

    for epoch in range(epochs):

        train_loss, train_acc = train_step(
            model = model,
            dataloader = train_dataloader,
            loss_fn = loss_fn,
            optimizer = optimizer
        )

        test_loss, test_acc = test_step(
            model = model,
            dataloader = test_dataloader,
            loss_fn = loss_fn
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)


        print(f"epoch: {epoch} | train loss: {train_loss:.3f} | train acc: {train_acc:.3f} | test loss: {test_loss:.3f} | test acc: {test_acc:.3f}")

    return results



Overwriting sources/engine.py


In [9]:
import torch
from sources import utils, datasetup, engine

# show directory tree
utils.list_file_tree()

# download data
url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
image_dir, train_dir, test_dir = utils.download_data(url=url)

# create train and test dataloader
BATCHSIZE = 32
NUMWORKERS = 1

train_transform, test_transform = datasetup.create_compose_transforms()

train_dataloader, test_dataloader, class_names = datasetup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    train_transform = train_transform,
    test_transform = test_transform,
    batch_size = BATCHSIZE,
    num_workers = NUMWORKERS
)

# create tinyVGG model
model_0 = engine.tinyVGG(
    name = "model_0",
    inp_shape = 3,
    out_shape = len(class_names),
    hidden_units = 10
)

# setup loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

# train model
model_0_results = engine.train(
    model=model_0,
    train_dataloader=train_dataloader,
    test_dataloader = test_dataloader,
    loss_fn=loss_fn,
    optimizer = optimizer,
    epochs=1
)

# save model
model_dir = Path("SAVED_MODEL")
model_dir.mkdir(parents=True, exist_ok=True)
model_saved_path = utils.save_model(
    model = model_0,
    model_dir = model_dir
)


  . 
     ./.config 
        ./.config/configurations 
        ./.config/logs 
           ./.config/logs/2023.10.18 
     ./sources 
        ./sources/__pycache__ 
     ./data 
        ./data/images 
           ./data/images/test 
              ./data/images/test/pizza 
              ./data/images/test/steak 
              ./data/images/test/sushi 
           ./data/images/train 
              ./data/images/train/pizza 
              ./data/images/train/steak 
              ./data/images/train/sushi 
     ./sample_data 
epoch: 0 | train loss: 1.114 | train acc: 0.305 | test loss: 1.112 | test acc: 0.260


In [14]:
loaded_model_0 = engine.tinyVGG(
    name = "loaded_model_0",
    inp_shape = 3,
    out_shape = len(class_names),
    hidden_units = 10
)

loaded_model_0.load_state_dict(torch.load(f = model_saved_path))

<All keys matched successfully>

In [16]:


loaded_model_1 = engine.tinyVGG(
    name = "loaded_model_1",
    inp_shape=3,
    out_shape=len(class_names),
    hidden_units = 10
)

load_saved_model(
    loaded_model = loaded_model_1,
    model_saved_path = model_saved_path
)